In [1]:
import csv
import os
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
from os.path import basename
import shutil
import numpy as np
import warnings
from tabulate import tabulate
from IPython.display import display, HTML
from multiprocessing import Pool
import tensorflow as tf

warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 2. Scraping Table 1

In [2]:
clinicalTrail = pd.read_csv('clinical_oa_df_checked.csv')

In [3]:
def table1(fileName, src='SampleFiles'):
    soup = BeautifulSoup(open(os.path.join(src, fileName)), 'lxml')
    tables = soup.find_all('table')
    dt = ['No Table1']
    tableStatus = 'No Table1'
    tabLen = len(tables)
    if tabLen> 0:
        i = 0
        while i < tabLen:
            table = tables[i]
            tableStatus = 'Has Table1'
            try:
                dfTmp = pd.read_html(str(table), na_values = ' ', keep_default_na= False)[0]
                nLevels = dfTmp.columns.nlevels
                colList = dfTmp.columns.tolist()
                if ((sum([('AGE' in colList[k][l].upper())])>0) | (
                        sum([('RACE' in colList[k][l].upper())])>0) | (
                        sum([(('MEN' in colList[k][l].upper()) | ('WOMEN' in colList[k][l].upper()) | 
                     ('SEX' in colList[k][l].upper()) | ('GENDER'in colList[k][l].upper()))]) > 0) for l 
                        in range(nLevels) for k in range(len(colList))):
                    dt = table
                    break
                stub = dfTmp.iloc[:, 0].apply(lambda x: x.upper())
                if ((sum([('AGE' in stub[i])])>0)|(sum([('RACE' in stub[i])])>0) |sum(
                    ('MEN' in stub[i])| ('WOMEN' in stub[i]) | ('SEX' in stub[i]) | (
                        'GENDER' in stub[i]))>0 
                    for i in range(len(stub))):
                    dt = table
                    break

            except ValueError:
                dt = ['Unable to parse Value']
                tableStatus = "Unable to parse Value"  

            except TypeError:
                dt = ['Unable to parse Type']
                tableStatus = "Unable to parse Type"  
            i += 1
    if dt == []:
        dt = ['Graph Table1']
        tableStatus = "Graphic Table1"
    clinicalTrail['ContainTable1'][clinicalTrail['Accession ID']== os.path.splitext(fileName)[0]] = tableStatus
    tableList[fileName] = dt

In [4]:
src = 'SampleFiles'
src_files = os.listdir(src)
clinicalTrail['ContainTable1'] = "No Table1"
tableList = {}

counter = 0
for file in src_files:
    counter += 1
    table1(file, src)
    if (counter % 100 == 0):
        print ("Files processed: ", counter )

Files processed:  100
Files processed:  200
Files processed:  300
Files processed:  400
Files processed:  500
Files processed:  600
Files processed:  700


In [5]:
clinicalTrail['ContainTable1'].unique()

array(['No Table1', 'Has Table1'], dtype=object)

In [6]:
clinicalTrail[clinicalTrail.ContainTable1 == "Has Table1"].count()

Unnamed: 0                            613
Unnamed: 0.1                          613
File                                  613
Article Citation                      613
Accession ID                          613
Last Updated (YYYY-MM-DD HH:MM:SS)    613
PMID                                  613
License                               613
Found                                 613
ContainTable1                         613
dtype: int64

In [7]:
(pd.DataFrame.from_dict(data=tableList, orient='index')
   .to_csv('table1List.csv', header=False))

In [8]:
clinicalTrail.to_csv("clinical_oa_df_table1.csv")